In [9]:
using Toms566
p = Problem(1)
x0 = p.x0;

In [10]:
function newtMin(p, x0)
    #set up all the variables
    k = 0;  #counter
    f = p.obj(x0);  #f is the objective value at point x
    g = p.grd(x0);  #g is the at point x
    H = p.hes(x0);  #H is the Hessian at point x
    tau = norm(g,2);  #calculate the initial 2-norm of the gradient at x0
    
    #We exit after |g| < tauGoal.  Here we scale tauGoal based on the initial value of tau
    if tau > 10e5
        tauGoal = tau*10e-10
    elseif tau > 10e3
        tauGoal = tau*10e-4
    elseif tau > 10e-3
        tauGoal = tau*10e-5
    else
        tauGoal = tau*10e-4
    end
    
    alpha = 1;  #a part of the step size for the newton step
    x = x0;  #setting the initial value of x
    mu = 10e-4  #this is for the Armijo backtracking linesearch, it helps define a better condition for an appropriate step size
    sigma = 10e-5  #this is the minimum value we will accept for an eigenvalue inside the Hessian
    
    #Main loop.  We exit when |g| is less than a predefined value (ie: we are near a stationary point)
    while tau > tauGoal
        #if we max iterations, exit and return the current value
        if k == 10e3
            @printf("Maxed iterations")
            return (k, p.obj(x))
        end
        
        #condition H so that it is definite positive under all conditions
        A = eigvals(H)
        V = eigvecs(H)
        for i in 1:1:length(A)
            if abs(A[i]) >= sigma
                A[i] = abs(A[i])
            else
                A[i] = sigma
            end
        end
        A = diagm(vec(A))
        H = *(*(V,A),transpose(V))
        
        #find search direction
        d = H\-(g)
        
        #Armijo Backtracking
        alpha = 1;
        while p.obj(x + d*alpha) > (p.obj(x) + alpha*mu*dot(g,d))
            alpha = alpha/1.5;
        end
        
        #compute next x
        x = x + alpha * d;
        
        #clean up for next loop
        f = p.obj(x);
        g = p.grd(x);
        H = p.hes(x);
        k = k + 1;
        tau = norm(g,2)
    end
    return (k, p.obj(x))  #return the value we've found
end

#run the minimizer
newtMin(p, x0)

(15,2.2717945476127312e-7)

In [134]:
#my test problem
type Blah5
    x0::Array{Float64}
    obj::Function
    grd::Function
    hes::Function
    
 
    function Blah5()
        this = new()
 
        this.x0 = [3,-3,10]
        
        #this function is u = x^2 + y^2 + z^3.  THe minimum should still be 0.
        this.obj = function (x::Array{Float64})
            return x[1]^2 + x[2]^2 + x[3]^3
        end
        
        #gradient
        this.grd = function (x::Array{Float64})
            return [2*x[1], 2*x[2], 3*x[3]^2]
        end
        
        #hessian
        this.hes = function (x::Array{Float64})
            return diagm([2.0,2.0,6.0*x[3]])
        end
 
        return this
    end
end

In [135]:
poo = Blah5()
x0 = poo.x0
newtMin(poo, x0)

(7,0.000476837158203125)